# ETL Accidentes de Tránsito Reportados en Antioquia

In [29]:
# %pip install pandas
# %pip install unidecode

In [30]:
#  Importar librerias
import pandas as pd
from unidecode import unidecode
import re

In [31]:
# Cargar datos
df = pd.read_csv("Accidentes_de_transito_reportados_en_Antioquia.csv")

In [32]:
# TRANSFORMACION Y LIMPIEZA DE DATOS
# FECHA ACCIDENTE
# CONVERTIR LA FECHA A FORMATO DATETIME
df["FECHA ACCIDENTE"] = pd.to_datetime(df["FECHA ACCIDENTE"], format="%d/%m/%Y")

# OBTENER AÑO, MES Y DIA
df["AÑO"] = df["FECHA ACCIDENTE"].dt.year
df["MES"] = df["FECHA ACCIDENTE"].dt.month
df["DIA"] = df["FECHA ACCIDENTE"].dt.day

# OBTENER DIA DE LA SEMANA
df["DIA DE LA SEMANA"] = df["FECHA ACCIDENTE"].dt.day_name()

# TRADUCIR EL DIA DE LA SEMANA A ESPAÑOL Y PASARLOS A MAYUSCULAS
dicc_espanol = {
    "Monday": "LUNES",
    "Tuesday": "MARTES",
    "Wednesday": "MIERCOLES",
    "Thursday": "JUEVES",
    "Friday": "VIERNES",
    "Saturday": "SABADO",
    "Sunday": "DOMINGO",
}

# MAPEAR LA COLUMNA DEL DIA DE LA SEMANA CON EL DICCIONARIO
df["DIA DE LA SEMANA"] = df["DIA DE LA SEMANA"].map(dicc_espanol)

In [33]:
# HORA ACCIDENTE
# CONVERTIR LA HORA A FORMATO TIME

df["HORA ACCIDENTE"] = pd.to_datetime(
    df["HORA ACCIDENTE"], format="%H:%M", errors="coerce"
).dt.time

In [34]:
# GRAVEDAD
# MAPEAR LOS VALORES DE LA GRAVEDAD

dicc_gravedad = {"d": "SOLO DAÑOS", "h": "HERIDOS", "m": "MUERTOS"}



df["GRAVEDAD"] = df["GRAVEDAD"].map(dicc_gravedad)

In [35]:
# CLASE ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES
df["CLASE ACCIDENTE"] = df["CLASE ACCIDENTE"].apply(unidecode)

In [36]:
# CAUSANTE ACCIDENTE
# PASAR A MAYUSCULAS LOS VALORES
df["CAUSANTE ACCIDENTE"] = df["CAUSANTE ACCIDENTE"].str.upper()

In [37]:
# DESCRIPCIÓN COLISIONANTE
# PASAR A MAYSUCULAS LOS VALORES Y ELIMINAR TILDES Y/O CARACTERES ESPECIALES
df["DESCRIPCIÓN COLISIONANTE"] = (
    df["DESCRIPCIÓN COLISIONANTE"].str.upper().apply(unidecode)
)

In [38]:
# DESCRIPCIÓN OBJETO FIJO
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y LOS VALORES DE NO REPORTADO YA QUE NO APLICAN PARA LA VISUALIZACIÓN DE LOS DATOS
df["DESCRIPCIÓN OBJETO FIJO"] = df["DESCRIPCIÓN OBJETO FIJO"].apply(unidecode).replace(
    "NO REPORTADO", pd.NA
)

In [39]:
# ZONA ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['ZONA ACCIDENTE'] = df['ZONA ACCIDENTE'].apply(unidecode).str.upper()

In [40]:
# DESCRIPCION DE ESTADO
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['DESCRIPCIÓN DE ESTADO'] = df['DESCRIPCIÓN DE ESTADO'].str.upper().apply(unidecode)


In [41]:
# ÁREA ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALE Y PASAR A MAYUSCULAS
df['ÁREA ACCIDENTE'] = df['ÁREA ACCIDENTE'].apply(unidecode).str.upper()

In [42]:
# SECTOR ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['SECTOR ACCIDENTE'] = df['SECTOR ACCIDENTE'].str.upper()

In [43]:
# DESCRIPCION LOCALIZACION
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['DESCRIPCIÓN LOCALIZACIÓN'] = df['DESCRIPCIÓN LOCALIZACIÓN'].apply(unidecode).str.upper()

In [44]:
# ESTADO CLIMA
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['ESTADO CLIMA'] = df['ESTADO CLIMA'].apply(unidecode).str.upper()


In [49]:
# MUNICIPIO
# FUNCION PARA LIMPIAR Y NORMALIZAR LOS NOMBRES DE LOS MUNICIPIOS
corrected_names = {
    "SANTA BARBARA - ANT": "SANTA BARBARA",
    "SANTABÁRBARA": "SANTA BARBARA",
    'STAFE DE ANTIOQUIA': 'SANTA FE DE ANTIOQUIA'
}

df["MUNICIPIO"] = df["MUNICIPIO"].replace(corrected_names).apply(unidecode)

array(['NO REPORTA', 'ABEJORRAL', 'SAN JERONIMO', 'PENOL', 'FREDONIA',
       'CAICEDO', 'SAN VICENTE', 'GUATAPE', 'AMAGA', 'SANTA BARBARA',
       'GUARNE', 'VENECIA', 'JARDIN', 'PUEBLO RICO', 'TAMESIS',
       'VALPARAISO', 'GRANADA', 'ALEJANDRIA', 'COCORNA', 'CARAMANTA',
       'BETANIA', 'SAN LUIS', 'TITIRIBI', 'BURITICA', 'SOPETRAN',
       'JERICO', 'LA UNION', 'ITUANGO', 'CONCORDIA', 'LIBORINA',
       'SANTA FE DE ANTIOQUIA', 'CACERES', 'ANZA', 'ANTIOQUIA'],
      dtype=object)

In [54]:
# DIRECCION LUGAR
# ELIMIAR TILDES Y/O CARACTERES ESPECIALES
df['DIRECCIÓN LUGAR'] = df['DIRECCIÓN LUGAR'].apply(unidecode)

In [59]:
# COLUMNAS RELACIONADAS A LAS VICTIMAS
# Verificación de valores nulos y reemplazo por 0 (si es apropiado)
columns_victims = ['NUMERO VICTIMA PEATÓN', 'NUMERO VICTIMA ACOMPAÑANTE', 'NUMERO VICTIMA PASAJERO', 'NUMERO VICTIMA CONDUCTOR', 'NUMERO VICTIMA HERIDO', 'NUMERO VICTIMA MUERTO']
df[columns_victims] = df[columns_victims].fillna(0)

# Verificación de valores negativos y corrección
for column in columns_victims:
    df[column] = df[column].apply(lambda x: max(x, 0))

# Suma total de víctimas por accidente
df['TOTAL VÍCTIMAS'] = df[columns_victims].sum(axis=1)